In [114]:
import matplotlib.pyplot as plt
import collections
import math

def vignere_encode(plaintext, key):
    plaintext = list(plaintext)
    key = list(key.lower())  
    
    for i in range(len(key)):
        key[i] = ord(key[i]) - 97
    
    for i in range(len(plaintext)):
        plaintext[i] = ord(plaintext[i])
        
    for i in range(len(plaintext)):
        plaintext[i] = (plaintext[i] + key[i % len(key)]) % 255
        
        if plaintext[i] > 90 and plaintext[i] < 97:
            plaintext[i] = ord('a') + (plaintext[i] - 91)
        if plaintext[i] > 122:
            plaintext[i] = ord('A') + (plaintext[i] - 123)
        
    for i in range(len(plaintext)):
        plaintext[i] = chr(plaintext[i])

    plaintext = "".join(plaintext)
    return plaintext

def vignere_decode(ciphertext, key):
    ciphertext = list(ciphertext)
    key = list(key.lower())  
    
    for i in range(len(key)):
        key[i] = ord(key[i]) - 97
    
    for i in range(len(ciphertext)):
        ciphertext[i] = ord(ciphertext[i])
        
    for i in range(len(ciphertext)):
        ciphertext[i] = (ciphertext[i] - key[i % len(key)]) % 255
        
        if ciphertext[i] < 65:
            ciphertext[i] = ord('z') - (64 - ciphertext[i])
        elif ciphertext[i] > 90 and ciphertext[i] < 97:
            ciphertext[i] = ord('Z') - (96 - ciphertext[i])
        
    for i in range(len(ciphertext)):
        ciphertext[i] = chr(ciphertext[i])

    ciphertext = "".join(ciphertext)
    return ciphertext

def plot_frequency_analysis(segment, letter_no):
    letter_counts = collections.Counter(segment)
    letters, counts = zip(*letter_counts.most_common())
    letters = [chr(i) for i in range(97, 123)]
    counts = [letter_counts[letter] for letter in letters]

    plt.title(f"Letter No: {letter_no} of the key")
    plt.bar(letters, counts)
    plt.show()

def predict_key_len(ciphertext):
    substrings = {}
    for i in range(len(ciphertext) - 2):
        substring = ciphertext[i : i + 2]
        if substring in substrings:
            substrings[substring].append(i)
        else:
            substrings[substring] = [i]
    
    distances = {}
    for substring in substrings:
        if len(substrings[substring]) > 1:
            for i in range(1, len(substrings[substring])):
                distance = substrings[substring][i] - substrings[substring][i - 1]
                if distance in distances:
                    distances[distance] += 1
                else:
                    distances[distance] = 1

    repeated_distances = []
    gcd_distances = math.factorial(50)
    for distance in distances:
        if distances[distance] > 1:
            repeated_distances.append(distance)
            if math.gcd(gcd_distances, distance) > 3:
                gcd_distances = math.gcd(gcd_distances, distance)
    # return gcd_distances
                
    common_multiple_count = [0 for i in range(0,100)]
    for i in range(4,100):
        for distance in repeated_distances:
            if distance % i == 0:
                common_multiple_count[i] += 1
    estimated_key_length = []
    for i in range(len(common_multiple_count)):
        estimated_key_length.append((common_multiple_count[i],i))
        
    estimated_key_length.sort(reverse=True)
    return_val = []
    for i in range(10):
        return_val.append(estimated_key_length[i][1])
    return return_val

def frequency_analysis(ciphertext, estimated_key_length):
    segments = [""] * estimated_key_length
    for i, char in enumerate(ciphertext):
        segments[i % estimated_key_length] += char
    key_guess = ""
    for segment in segments:
        segment_freq = collections.Counter(segment)
        segment_freq = dict(sorted(segment_freq.items()))
        total_count = sum(segment_freq.values())
        segment_freq = {k: v / total_count * 100 for k, v in segment_freq.items()}
        
        # plot_frequency_analysis(segment, segments.index(segment))
        
        max_freq, max_freq_letter = 0, ''
        for letter in segment_freq:
            if segment_freq[letter] > max_freq:
                max_freq = segment_freq[letter]
                max_freq_letter = letter
        
        key_letter = chr((ord(max_freq_letter) - 101) + 97)
        key_guess += key_letter
        
    return key_guess



def prediction_match(input,predicted):
    with open(input, "r") as f:
        input_text = f.read()
    with open(predicted, "r") as f:
        predicted_text = f.read()
    match = 0
    for i in range(len(input_text)):
        if input_text[i] == predicted_text[i]:
            match += 1
    match_percentage = (match/len(input_text))*100
    return match_percentage

def vignere_crack_kasiski(ciphertext):
    estimated_key_len = predict_key_len(ciphertext)
    print("Estimated Key Length:", estimated_key_len)
    for est_key_len in estimated_key_len:
        key_guess = frequency_analysis(ciphertext, est_key_len)
        print("Guessed Key:", key_guess)
        plaintext = vignere_decode(ciphertext, key_guess)
        with open(f"prediction/predicted_with_len{est_key_len}.txt", "w") as f:
            f.write(plaintext)
        print("Prediction Match:", prediction_match("input.txt", f"prediction/predicted_with_len{est_key_len}.txt"))


def main():
    plaintext = ""
    key = ""
    
    with open("key.txt", "r") as f:
        key = f.read().strip()
    with open("input.txt", "r") as f:
        for line in f:
            plaintext += line.strip()
    
    ciphertext = vignere_encode(plaintext, key)
    back_to_plaintext = vignere_decode(ciphertext, key)
    # print(back_to_plaintext)
    with open("encoded.txt", "w") as f:
        f.write(ciphertext)
        
    vignere_crack_kasiski(ciphertext)

if __name__ == "__main__":
    main()

Estimated Key Length: [4, 6, 5, 9, 8, 18, 7, 10, 12, 14]
Guessed Key: o?o?
Prediction Match: 26.30898384658232
Guessed Key: pus?o?
Prediction Match: 15.648126044402005
Guessed Key: ooooo
Prediction Match: 11.162170764701202
Guessed Key: Io>o?okk?
Prediction Match: 32.29688867669292
Guessed Key: o?o?o?p?
Prediction Match: 23.517943821118802
Guessed Key: thisisake?oka?oka?
Prediction Match: 98.31901010583273
Guessed Key: oooosoo
Prediction Match: 11.154213416089759
Guessed Key: o?ouo?o?p?
Prediction Match: 21.051165751571578
Guessed Key: pus?o?p?o?o?
Prediction Match: 21.005410997055783
Guessed Key: o?oup?o?o?o?ou
Prediction Match: 20.380759131057534
